# Adapatar dataset de treino para treinar com Triplet Loss

Isso é devido a estrutura esperada por cada tipo de loss:

| **Tipo de Loss**         | **Estrutura esperada**                    | **Exemplo**                                                               |
|-----------------------|---------------------------------------|-----------------------------------------------------------------------|
| CosineSimilarityLoss  | `pares` (sent1, sent2, score)           | “O gato dorme.” / “O felino está dormindo.” → 0.9                     |
| TripletLoss	        | `triplas` (anchor, positive, negative)  | “O gato dorme.” / “O felino está dormindo.” / “O cachorro correu.”    |

Ou seja:
- **Cosine** aprende graus de similaridade contínuos (0–5, ou 0–1).
- **Triplet** aprende relações relativas (“A é mais parecido com B do que com C”).

In [5]:
import pandas as pd
import random

df = pd.read_csv("../data/train.csv")

# Pares positivos: label == 1
# Pares negativos: label == 0
positivos = df[df['label'] == 1]
negativos = df[df['label'] == 0]

triplets = []
for _, row in positivos.iterrows():
    anchor = row['question']
    positive = row['answer']

    # escolhe um negativo aleatório
    neg = negativos.sample(1).iloc[0]['answer']
    triplets.append((anchor, positive, neg))

df_triplet = pd.DataFrame(triplets, columns=['anchor', 'positive', 'negative'])
df_triplet.to_csv("../data/train_triplet.csv", index=False)

print(f"✅ Geradas {len(df_triplet)} triplas.")

✅ Geradas 348 triplas.


## Turbinando dados de treino

Criando `k` triplas por par positivo (label==1).

In [7]:
import pandas as pd
df = pd.read_csv("../data/train.csv")
positivos = df[df['label']==1].copy()
negativos = df[df['label']==0]['answer'].tolist()

k = 5
triplets = []
for _, row in positivos.iterrows():
    A, P = row['question'], row['answer']
    for _ in range(k):
        N = random.choice(negativos)
        if N != A and N != P:
            triplets.append((A, P, N))

pd.DataFrame(triplets, columns=['anchor','positive','negative']).to_csv(
    "../data/train_triplet.csv", index=False
)
print(f"> Total de {len(triplets)} triplas.")

> Total de 1740 triplas.
